### In this file, we will reconstruct our data into graphs

Instructions: You should probably have another separate notebook that creates the graph version of the dataset. Again, you should save the data, and for this make use to use the save_graph and load_graph functions of DGL.

Important Libraries:
ase
Structure/Geometry of a molecule